In [1]:
# Set up Environment
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
import matplotlib.pyplot as plt

from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report, precision_recall_curve
import warnings

# Ignore warnings
warnings.filterwarnings("ignore")

In [2]:
#Importing DataFrame
df = pd.read_csv("/home/achoo/Desktop/Honeypot/Train_sample.txt",sep=",",names=["duration","protocoltype","service","flag","srcbytes","dstbytes","land", "wrongfragment","urgent","hot","numfailedlogins","loggedin", "numcompromised","rootshell","suattempted","numroot","numfilecreations", "numshells","numaccessfiles","numoutboundcmds","ishostlogin",
"isguestlogin","count","srvcount","serrorrate", "srvserrorrate",
"rerrorrate","srvrerrorrate","samesrvrate", "diffsrvrate", "srvdiffhostrate","dsthostcount","dsthostsrvcount","dsthostsamesrvrate", "dsthostdiffsrvrate","dsthostsamesrcportrate",
"dsthostsrvdiffhostrate","dsthostserrorrate","dsthostsrvserrorrate",
"dsthostrerrorrate","dsthostsrvrerrorrate","attack", "lastflag"])
df.head()

,duration,protocoltype,service,flag,srcbytes,dstbytes,land,wrongfragment,urgent,hot,...,dsthostsamesrvrate,dsthostdiffsrvrate,dsthostsamesrcportrate,dsthostsrvdiffhostrate,dsthostserrorrate,dsthostsrvserrorrate,dsthostrerrorrate,dsthostsrvrerrorrate,attack,lastflag
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal,20
1,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15
2,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19
3,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21
4,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21


In [3]:
# Show DataFram
df.describe()

,duration,srcbytes,dstbytes,land,wrongfragment,urgent,hot,numfailedlogins,loggedin,numcompromised,...,dsthostsrvcount,dsthostsamesrvrate,dsthostdiffsrvrate,dsthostsamesrcportrate,dsthostsrvdiffhostrate,dsthostserrorrate,dsthostsrvserrorrate,dsthostrerrorrate,dsthostsrvrerrorrate,lastflag
count,70000.000000,7.000000e+04,7.000000e+04,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,...,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000
mean,290.443114,4.954804e+04,2.179300e+04,0.000186,0.022657,0.000057,0.195243,0.001157,0.393557,0.327114,...,115.110071,0.519420,0.083222,0.148575,0.032641,0.285607,0.279635,0.119190,0.120417,19.492800
std,2617.509680,5.828998e+06,4.951689e+06,0.013627,0.253944,0.011952,2.099391,0.045183,0.488542,30.260462,...,110.590412,0.449026,0.189012,0.309723,0.112991,0.445287,0.446178,0.307004,0.319508,2.309996
min,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,10.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,18.000000
50%,0.000000,4.400000e+01,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,62.000000,0.500000,0.030000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,20.000000
75%,0.000000,2.760000e+02,5.010000e+02,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,255.000000,1.000000,0.070000,0.060000,0.020000,1.000000,1.000000,0.000000,0.000000,21.000000
max,42908.000000,1.167519e+09,1.309937e+09,1.000000,3.000000,3.000000,77.000000,4.000000,1.000000,7479.000000,...,255.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,21.000000


In [4]:
#Removing specific rows from Data
df.drop(['land','urgent','numfailedlogins','numoutboundcmds'],axis=1,inplace=True)
#Check for missig sum
df.isna().sum()

duration                  0
protocoltype              0
service                   0
flag                      0
srcbytes                  0
dstbytes                  0
wrongfragment             0
hot                       0
loggedin                  0
numcompromised            0
rootshell                 0
suattempted               0
numroot                   0
numfilecreations          0
numshells                 0
numaccessfiles            0
ishostlogin               0
isguestlogin              0
count                     0
srvcount                  0
serrorrate                0
srvserrorrate             0
rerrorrate                0
srvrerrorrate             0
samesrvrate               0
diffsrvrate               0
srvdiffhostrate           0
dsthostcount              0
dsthostsrvcount           0
dsthostsamesrvrate        0
dsthostdiffsrvrate        0
dsthostsamesrcportrate    0
dsthostsrvdiffhostrate    0
dsthostserrorrate         0
dsthostsrvserrorrate      0
dsthostrerrorrate   

In [5]:
#Exclude colums with data tyoes that aren't numbers
df.select_dtypes(exclude=[np.number])

,protocoltype,service,flag,attack
0,tcp,ftp_data,SF,normal
1,udp,other,SF,normal
2,tcp,private,S0,neptune
3,tcp,http,SF,normal
4,tcp,http,SF,normal
...,...,...,...,...
69995,icmp,ecr_i,SF,smurf
69996,tcp,supdup,S0,neptune
69997,icmp,ecr_i,SF,smurf
69998,tcp,http,SF,normal


In [6]:
# Replace all values in 'attack' column that are not 'normal' with 'attack'
df['attack'].loc[df['attack']!='normal']='attack'

# Initialize a LabelEncoder to encode categorical columns
le=LabelEncoder()

In [7]:
# Encode the columns (convert categorical data to numeric)
df['protocoltype']=le.fit_transform(df['protocoltype'])
df['service']=le.fit_transform(df['service'])
df['flag']=le.fit_transform(df['flag'])
df['attack']=le.fit_transform(df['attack'])

In [8]:
# Separate features (X) and labels (y)
X=df.drop(['attack'],axis=1)
y=df['attack']

# Initialize StandardScaler to scale feature data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
#X_transformed = scaler.transform(X)

# Split data to train and validate sets
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

In [9]:
###Using Random Forest Classifier
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

#Predicttions on validation set
rf_pred=rf.predict(X_val)

#Create DataFrame for predictions
rf_df = pd.DataFrame()
rf_df['actual'] = y_val
rf_df['pred'] = rf_pred
rf_df.head()

,actual,pred
47285,1,1
38115,0,0
7761,0,0
42784,1,1
30545,1,1


In [10]:
#Print Evaluation
print("Random Forest Accuracy Score: ", accuracy_score(y_val, rf_pred))
print("Random Forest Confusion Matrix:\n", confusion_matrix(y_val, rf_pred))
print("Random Forest Classification Report:\n", classification_report(y_val, rf_pred))

Random Forest Accuracy Score:  0.9996428571428572
Random Forest Confusion Matrix:
 [[6553    0]
 [   5 7442]]
Random Forest Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6553
           1       1.00      1.00      1.00      7447

    accuracy                           1.00     14000
   macro avg       1.00      1.00      1.00     14000
weighted avg       1.00      1.00      1.00     14000



In [11]:
# Train usinng XGBoostModel
xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
xgb.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...)

In [12]:
# Predictions on validation set
y_val_pred = xgb.predict(X_val)

# Print evaluation metrics
print("XGBoost Classification Report:\n", classification_report(y_val, y_val_pred))
print("Confusion Matrix:\n", confusion_matrix(y_val, y_val_pred))

XGBoost Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6553
           1       1.00      1.00      1.00      7447

    accuracy                           1.00     14000
   macro avg       1.00      1.00      1.00     14000
weighted avg       1.00      1.00      1.00     14000

Confusion Matrix:
 [[6549    4]
 [   5 7442]]


In [13]:
###COMBINED MODELS###

#Create a VotingClassifier with the defined models
voting_clf = VotingClassifier(estimators=[
    ('rf', rf),
    ('xgb', xgb)
], voting='hard')

In [14]:
# Fit the VotingClassifier on the training data
voting_clf.fit(X_train, y_train)

# Predictions on the validation set
y_val_pred_voting = voting_clf.predict(X_val)

In [15]:
# Print evaluation metrics for the voting classifier
print("Voting Classifier Validation Classification Report:\n", classification_report(y_val, y_val_pred_voting))
print("Confusion Matrix:\n", confusion_matrix(y_val, y_val_pred_voting))

Voting Classifier Validation Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6553
           1       1.00      1.00      1.00      7447

    accuracy                           1.00     14000
   macro avg       1.00      1.00      1.00     14000
weighted avg       1.00      1.00      1.00     14000

Confusion Matrix:
 [[6553    0]
 [   6 7441]]


In [16]:
### TESTING SET ###
test_df = pd.read_csv("/home/achoo/Desktop/Honeypot/Test_sample.txt",sep=",",names=["duration","protocoltype","service","flag","srcbytes","dstbytes","land", "wrongfragment","urgent","hot","numfailedlogins","loggedin", "numcompromised","rootshell","suattempted","numroot","numfilecreations", "numshells","numaccessfiles","numoutboundcmds","ishostlogin",
"isguestlogin","count","srvcount","serrorrate", "srvserrorrate",
"rerrorrate","srvrerrorrate","samesrvrate", "diffsrvrate", "srvdiffhostrate","dsthostcount","dsthostsrvcount","dsthostsamesrvrate", "dsthostdiffsrvrate","dsthostsamesrcportrate",
"dsthostsrvdiffhostrate","dsthostserrorrate","dsthostsrvserrorrate",
"dsthostrerrorrate","dsthostsrvrerrorrate","attack", "lastflag"])
test_df.head()

,duration,protocoltype,service,flag,srcbytes,dstbytes,land,wrongfragment,urgent,hot,...,dsthostsamesrvrate,dsthostdiffsrvrate,dsthostsamesrcportrate,dsthostsrvdiffhostrate,dsthostserrorrate,dsthostsrvserrorrate,dsthostrerrorrate,dsthostsrvrerrorrate,attack,lastflag
0,0,tcp,private,REJ,0,0,0,0,0,0,...,0.04,0.06,0.00,0.00,0.0,0.0,1.00,1.00,neptune,21
1,0,tcp,private,REJ,0,0,0,0,0,0,...,0.00,0.06,0.00,0.00,0.0,0.0,1.00,1.00,neptune,21
2,2,tcp,ftp_data,SF,12983,0,0,0,0,0,...,0.61,0.04,0.61,0.02,0.0,0.0,0.00,0.00,normal,21
3,0,icmp,eco_i,SF,20,0,0,0,0,0,...,1.00,0.00,1.00,0.28,0.0,0.0,0.00,0.00,saint,15
4,1,tcp,telnet,RSTO,0,15,0,0,0,0,...,0.31,0.17,0.03,0.02,0.0,0.0,0.83,0.71,mscan,11


In [17]:
#Removing specific rows from Data
test_df.drop(['land','urgent','numfailedlogins','numoutboundcmds'],axis=1,inplace=True)

# Replace all values in 'attack' column that are not 'normal' with 'attack'
test_df['attack'].loc[test_df['attack']!='normal']='attack'

#Exclude colums with data tyoes that aren't numbers
test_df.select_dtypes(exclude=[np.number])

,protocoltype,service,flag,attack
0,tcp,private,REJ,attack
1,tcp,private,REJ,attack
2,tcp,ftp_data,SF,normal
3,icmp,eco_i,SF,attack
4,tcp,telnet,RSTO,attack
...,...,...,...,...
14995,tcp,http,SF,normal
14996,icmp,ecr_i,SF,normal
14997,tcp,supdup,S0,attack
14998,tcp,http,SF,normal


In [18]:
# Encode the columns (convert categorical data to numeric)
test_df['protocoltype']=le.fit_transform(test_df['protocoltype'])
test_df['service']=le.fit_transform(test_df['service'])
test_df['flag']=le.fit_transform(test_df['flag'])
test_df['attack']=le.fit_transform(test_df['attack'])

In [19]:
# Separate features (X) and labels (y)
X_test = test_df.drop(['attack'], axis=1)
y_test = test_df['attack']

# Scale the test features
X_test_scaled = scaler.transform(X_test)

In [20]:
###Using Random Forest on Test Data
#X_test_transformed = scaler.transform(X_test)
test_pred = rf.predict(X_test_scaled)

rf_test_df = pd.DataFrame()
rf_test_df['actual'] = y_test
rf_test_df['pred'] = test_pred
rf_test_df.head()

,actual,pred
0,0,0
1,0,0
2,1,1
3,0,0
4,0,1


In [21]:
# Print evaluation metrics for Random Forest
print("Accuracy Score: ", accuracy_score(y_test, test_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, test_pred))
print("Random Forest Classification Report:\n", classification_report(y_test, test_pred))

Accuracy Score:  0.8338
Confusion Matrix:
 [[6237 2319]
 [ 174 6270]]
Random Forest Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.73      0.83      8556
           1       0.73      0.97      0.83      6444

    accuracy                           0.83     15000
   macro avg       0.85      0.85      0.83     15000
weighted avg       0.87      0.83      0.83     15000



In [22]:
# Initialize and fit LOF
lof = LocalOutlierFactor()
y_test_pred_lof = lof.fit_predict(X_test_scaled)
y_test_pred_lof_binary = [0 if x == 1 else 1 for x in y_test_pred_lof]

In [23]:
# Print test evaluation metrics
y_test_pred = xgb.predict(X_test_scaled)
print("XGBoost Test Classification Report:\n", classification_report(y_test, y_test_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_test_pred))

XGBoost Test Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.76      0.85      8556
           1       0.75      0.97      0.85      6444

    accuracy                           0.85     15000
   macro avg       0.86      0.87      0.85     15000
weighted avg       0.88      0.85      0.85     15000

Confusion Matrix:
 [[6516 2040]
 [ 182 6262]]


In [24]:
###COMBINED MODELS###
# Predictions on the test set
y_test_pred = voting_clf.predict(X_test_scaled)

In [25]:
# Evaluation on the test set
print("Voting Classifier Test Classification Report:\n", classification_report(y_test, y_test_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_test_pred))

Voting Classifier Test Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.78      0.87      8556
           1       0.77      0.97      0.86      6444

    accuracy                           0.86     15000
   macro avg       0.87      0.88      0.86     15000
weighted avg       0.89      0.86      0.86     15000

Confusion Matrix:
 [[6674 1882]
 [ 183 6261]]
